# Study Hunalign

## Step 1: Concate bible text and tokenize

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from bible_util import *
import pandas as pd

file = "bible_sentences_ctb.csv"
sents = read_bible_sentences(file)
chaps = get_all_chaps(sents.index)

In [27]:
chaps_train, chaps_test = train_test_split(chaps, test_size=0.1, random_state=33)
sents_test = sents_filter_chaps(sents, chaps_test)
sents_train = sents_filter_chaps(sents, chaps_train)

## Need to generate:
* .raw file for raw text
* .stem file for sentences
* .manual.ladder for ground truth


In [28]:
index_bo = prepare_align_bo(sents_train, "bible_ctb_train.bo")

Rows processed... 1000
Rows processed... 2000
Rows processed... 3000
Rows processed... 4000
Rows processed... 5000
Rows processed... 6000
Rows processed... 7000


In [29]:
index_bo_verse = prepare_align_bo(sents_train, "bible_ctb_train_ver.bo", no_break_verse=True)

Rows processed... 1000
Rows processed... 2000
Rows processed... 3000
Rows processed... 4000
Rows processed... 5000
Rows processed... 6000
Rows processed... 7000


In [32]:
file = "bible_sentences_nirv.csv"
sents = read_bible_sentences(file)
sents_train = sents_filter_chaps(sents, chaps_train)

In [33]:
index_en = prepare_align_en(sents_train, file_prefix = "bible_nirv_train.en")

Rows processed... 1000
Rows processed... 2000
Rows processed... 3000
Rows processed... 4000
Rows processed... 5000
Rows processed... 6000
Rows processed... 7000


In [38]:
index_en_verse = prepare_align_en(sents_train, file_prefix = "bible_nirv_train_ver.en", no_break_verse=True)

Rows processed... 1000
Rows processed... 2000
Rows processed... 3000
Rows processed... 4000
Rows processed... 5000
Rows processed... 6000
Rows processed... 7000


In [40]:
print(len(index_bo), len(index_en))

17163 20498


## Create manual ladder

In [43]:
verse_index = f7(index_en)
write_ladder_file("bible_nirv_ctb.manual.ladder", index_bo, index_en, verse_index)
write_ladder_file("bible_nirv_ctb.manual2.ladder", index_bo, index_en_verse, verse_index)
write_ladder_file("bible_nirv_ctb.manual3.ladder", index_bo_verse, index_en_verse, verse_index)

## Run!

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train.en.stem  bible_ctb_train.bo.stem  -hand=bible_nirv_ctb.manual.ladder > bible_ctb_train_align.txt
13359 misaligned out of 7223 correct items, 16957 bets.
Precision: 0.212184, Recall: 0.498131
Quality 0.155236

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train.en.stem  bible_ctb_train.bo.stem  -hand=bible_nirv_ctb.manual.ladder -realign > bible_ctb_train_align.txt
12350 misaligned out of 7223 correct items, 17048 bets.
Precision: 0.275575, Recall: 0.650422
Quality 0.491821

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train_ver.en.stem  bible_ctb_train.bo.stem  -hand=bible_nirv_ctb.manual2.ladder > bible_ctb_train_align.txt
5976 misaligned out of 7223 correct items, 9783 bets.
Precision: 0.389144, Recall: 0.527066
Quality 0.00536566

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train_ver.en.stem  bible_ctb_train.bo.stem  -hand=bible_nirv_ctb.manual2.ladder -realign > bible_ctb_train_align.txt
4544 misaligned out of 7223 correct items, 9459 bets.
Precision: 0.519611, Recall: 0.680465
Quality 0.199176

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train_ver.en.stem  bible_ctb_train_ver.bo.stem  -hand=bible_nirv_ctb.manual3.ladder > bible_ctb_train_align.txt
267 misaligned out of 7223 correct items, 7397 bets.
Precision: 0.963904, Recall: 0.987124
Quality 0.387003

hunalign/src/hunalign/hunalign monlam_bo_en.dic bible_nirv_train_ver.en.stem  bible_ctb_train_ver.bo.stem  -hand=bible_nirv_ctb.manual3.ladder -realign > bible_ctb_train_align.txt
239 misaligned out of 7223 correct items, 7391 bets.
Precision: 0.967663, Recall: 0.99017
Quality 0.797119


## Dict (this part can run standalone way).

In [4]:
from pystardict import Dictionary
d = Dictionary("monlam-dictionary-stardict/Tibetan-English/tben")
for key in d.keys():
    print(key, d[key])
    break
with open("monlam_bo_en.dic", 'w') as f:
    for key in d.keys():
        key_tok = tokenize(key).strip()
        for target in d[key].split(", "):
            f.write(key_tok + " @ " + target + "\n")

ཀ་ one, pillar, all, the, the very, the beginning, the root, power, all the, indeed, surely, SA ka ba, symbol of origin of reality, infinitive particle


/usr/local/lib/python3.8/site-packages/pystardict.py:601: UserWarning: Iter dict items with in_memory=False may cause serious performance problem
  warnings.warn(
/usr/local/lib/python3.8/site-packages/pystardict.py:200: UserWarning: Iter dict items with in_memory=False may cause serious performance problem
  warnings.warn(


# conclusion: 

The hunalign gives really good result. Metrics: the precision and coverage is measure on rungs. Quality metrics is mainly the overlapping tokens normalized by long sentence length.

TODO:
* may need to fine tue trivialTranslateWord because the different script used